# 원, 삼각형, 사각형 구분하기
- 어려운 데이터 문제를 데이터 부풀리기를 이용한 경우

In [1]:
import numpy as np
import tensorflow as tf
seed=2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터 부풀리기

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.5,
    zoom_range=[0.8,2.0],
    vertical_flip=True,
    fill_mode='nearest'
    )


In [6]:
x.shape

(24, 24, 3)

In [7]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
img = load_img('handwriting/hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [8]:
x.shape

(1, 24, 24, 3)

In [9]:
i=0
for batch in train_datagen.flow(x,batch_size=1, save_to_dir='preview', save_prefix='tri', save_format='png'):
    i += 1
    if i > 20:
        break

In [10]:
train_generator = train_datagen.flow_from_directory(
    'handwriting/hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [11]:
test_datagen = ImageDataGenerator(rescale=1/255.)
test_generator = test_datagen.flow_from_directory(
    'handwriting/hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [4]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [5]:
train_generator.filenames[0]

'circle\\circle001.png'

## 모델 정의/설정

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense 

In [13]:
model = Sequential([
    Conv2D(32, (3,3), input_shape=(24,24,3), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [14]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## 모델 학습
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 test_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [16]:
model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=200,
    validation_data=test_generator,
    validation_steps=5, verbose=2
)

Epoch 1/200
15/15 - 0s - loss: 1.0423 - accuracy: 0.4667 - val_loss: 1.0783 - val_accuracy: 0.4667
Epoch 2/200
15/15 - 0s - loss: 0.8786 - accuracy: 0.6222 - val_loss: 1.0688 - val_accuracy: 0.4000
Epoch 3/200
15/15 - 0s - loss: 0.6611 - accuracy: 0.6444 - val_loss: 1.2021 - val_accuracy: 0.6000
Epoch 4/200
15/15 - 0s - loss: 0.4868 - accuracy: 0.7556 - val_loss: 1.8174 - val_accuracy: 0.5333
Epoch 5/200
15/15 - 0s - loss: 0.4895 - accuracy: 0.8222 - val_loss: 1.2654 - val_accuracy: 0.6667
Epoch 6/200
15/15 - 0s - loss: 0.6438 - accuracy: 0.8222 - val_loss: 0.9337 - val_accuracy: 0.6667
Epoch 7/200
15/15 - 0s - loss: 0.5454 - accuracy: 0.8000 - val_loss: 1.3278 - val_accuracy: 0.5333
Epoch 8/200
15/15 - 0s - loss: 0.6484 - accuracy: 0.7556 - val_loss: 0.8655 - val_accuracy: 0.6000
Epoch 9/200
15/15 - 0s - loss: 0.2830 - accuracy: 0.8889 - val_loss: 1.5257 - val_accuracy: 0.5333
Epoch 10/200
15/15 - 0s - loss: 0.2734 - accuracy: 0.8222 - val_loss: 1.2415 - val_accuracy: 0.5333
Epoch 11/

## 모델 평가

In [17]:
model.evaluate(test_generator, steps=5)

5/5 [==============================] - 0s 2ms/step - loss: 2.5508 - accuracy: 0.8000


[2.550828218460083, 0.800000011920929]